# Wormgear 3D Geometry Generation on Google Colab

This notebook generates STEP files for worm gears using build123d.

**Note:** We use condacolab to create a clean environment that avoids IPython version conflicts.

In [ ]:
# Install condacolab (creates isolated conda environment)
!pip install -q condacolab
import condacolab
condacolab.install()  # This will restart the runtime

In [ ]:
# After runtime restart, install build123d via conda-forge
# This gets OCP (OpenCascade) binaries that work on Linux
!mamba install -y -c conda-forge build123d

In [ ]:
# Install wormgear (calculator + CLI)
!pip install wormgear --no-deps
!pip install click  # CLI dependency

In [ ]:
# Test that build123d works
from build123d import Box, export_step
test_box = Box(10, 10, 10)
print(f"build123d working! Test box volume: {test_box.volume:.1f} mm³")

In [ ]:
# Import wormgear
from wormgear.calculator import design_from_module, validate_design
from wormgear.calculator.output import to_json
from wormgear.core import WormGeometry, WheelGeometry, BoreFeature, KeywayFeature
import json

## Design the Worm Gear Set

In [ ]:
# Create a design
design = design_from_module(
    module=2.0,
    ratio=30,
    num_starts=1,
    pressure_angle=20.0,
    hand="right"
)

# Validate
validation = validate_design(design)
print(f"Design valid: {validation.valid}")
print(f"Worm OD: {design.worm.tip_diameter_mm:.2f} mm")
print(f"Wheel OD: {design.wheel.tip_diameter_mm:.2f} mm")
print(f"Centre distance: {design.assembly.centre_distance_mm:.2f} mm")

## Generate 3D Geometry

In [ ]:
# Generate worm geometry
print("Generating worm...")
worm_geo = WormGeometry(
    params=design.worm,
    assembly_params=design.assembly,
    length=40.0,  # Worm length in mm
    bore=BoreFeature(diameter=8.0),
    keyway=KeywayFeature()
)
worm = worm_geo.build()
print(f"Worm generated! Volume: {worm.volume:.1f} mm³")

In [ ]:
# Generate wheel geometry
print("Generating wheel...")
wheel_geo = WheelGeometry(
    params=design.wheel,
    worm_params=design.worm,
    assembly_params=design.assembly,
    bore=BoreFeature(diameter=12.0),
    keyway=KeywayFeature()
)
wheel = wheel_geo.build()
print(f"Wheel generated! Volume: {wheel.volume:.1f} mm³")

## Export STEP Files

In [ ]:
# Export to STEP format
worm_geo.export_step("worm_m2_z1.step")
wheel_geo.export_step("wheel_m2_z30.step")

print("Exported:")
print("  - worm_m2_z1.step")
print("  - wheel_m2_z30.step")

In [ ]:
# Also save the design JSON
design_json = to_json(design)
with open('design.json', 'w') as f:
    json.dump(design_json, f, indent=2)
print("Saved design.json")

In [ ]:
# Download files (Colab-specific)
from google.colab import files

print("Downloading STEP files...")
files.download('worm_m2_z1.step')
files.download('wheel_m2_z30.step')
files.download('design.json')

## Alternative: Use the CLI

In [ ]:
# You can also use the CLI
!wormgear-geometry design.json --worm-bore 8 --wheel-bore 12

In [ ]:
# List generated files
!ls -la *.step *.json